In [1]:
import sqlite3
import pandas as pd
import logging
import os
from datetime import datetime, timedelta


In [2]:
user_activity = pd.read_csv("user_activity.csv", parse_dates=["last_login_date"])
transactions = pd.read_csv("transactions.csv", parse_dates=["transaction_date"])
user_preferences = pd.read_csv("user_preferences.csv")

In [3]:
# Validate Required Columns
# -----------------------------
def validate_columns(df, required_cols, name):
    missing = set(required_cols) - set(df.columns)
    if missing:
        raise ValueError(f" Missing columns in {name}: {missing}")
    print(f" {name} validated.")

validate_columns(user_activity, ["user_id", "last_login_date", "feature_usage_count", "account_type"], "user_activity")
validate_columns(transactions, ["transaction_id", "user_id", "transaction_date", "amount", "platform"], "transactions")
validate_columns(user_preferences, ["user_id", "communication_preference", "interface_theme", "notification_settings"], "user_preferences")

 user_activity validated.
 transactions validated.
 user_preferences validated.


In [4]:
# Create in-memory SQLite database
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()


In [5]:
# Write dataframes to SQL tables
user_activity.to_sql("user_activity", conn, index=False, if_exists="replace")
transactions.to_sql("transactions", conn, index=False, if_exists="replace")
user_preferences.to_sql("user_preferences", conn, index=False, if_exists="replace")

logging.info(" Data loaded into SQLite memory database.")


In [7]:
# Step 1: Identify active users
active_users = pd.read_sql_query("""
    SELECT *
    FROM user_activity
    WHERE DATE(last_login_date) >= DATE('now', '-30 days')
""", conn)

print(active_users)

  user_id      last_login_date  feature_usage_count account_type
0      u1  2025-07-25 00:00:00                   20      premium
1      u3  2025-07-18 00:00:00                   12      premium
2      u4  2025-07-01 00:00:00                    2        basic
3      u6  2025-07-22 00:00:00                   17   enterprise
4      u7  2025-07-10 00:00:00                    9      premium


In [10]:
# #Step 2: High-value customers among active users (top 20% spenders)
top20_spending_query = pd.read_sql_query("""
WITH user_spending AS (
    SELECT 
        user_id,
        SUM(amount) AS total_spend
    FROM transactions
    GROUP BY user_id
),
spending_stats AS (
    SELECT 
        total_spend
    FROM user_spending
    ORDER BY total_spend DESC
),
threshold AS (
    SELECT 
        MIN(total_spend) AS top_20_threshold
    FROM (
        SELECT total_spend
        FROM spending_stats
        LIMIT CAST((SELECT COUNT(*) * 0.2 FROM user_spending) AS INT)
    )
)
SELECT us.*
FROM user_spending us
JOIN threshold t ON us.total_spend >= t.top_20_threshold
ORDER BY us.total_spend DESC;
""", conn)

print(top20_spending_query)


  user_id  total_spend
0      u6          700


In [15]:
#Step 3: User preference trends for high-value users
#-- Aggregate user preferences
user_pref_trends_query = pd.read_sql_query("""
WITH user_spending AS (
    SELECT 
        user_id,
        SUM(amount) AS total_spend
    FROM transactions
    GROUP BY user_id
),
spending_stats AS (
    SELECT 
        total_spend
    FROM user_spending
    ORDER BY total_spend DESC
),
threshold AS (
    SELECT 
        MIN(total_spend) AS top_20_threshold
    FROM (
        SELECT total_spend
        FROM spending_stats
        LIMIT CAST((SELECT COUNT(*) * 0.2 FROM user_spending) AS INT)
    )
),
top_customers AS (
    SELECT us.user_id
    FROM user_spending us
    JOIN threshold t ON us.total_spend >= t.top_20_threshold
)
SELECT 
    up.communication_preference,
    up.interface_theme,
    up.notification_settings,
    COUNT(*) AS user_count
FROM top_customers tc
JOIN user_preferences up ON tc.user_id = up.user_id
GROUP BY 
    up.communication_preference,
    up.interface_theme,
    up.notification_settings
ORDER BY user_count DESC;
""", conn)

print(user_pref_trends_query)



  communication_preference interface_theme notification_settings  user_count
0                    email            dark              standard           1


In [16]:
conn.close()